# Pysam

Pysam è un package che mette a disposizione le funzionalità per manipolare file in formato SAM/BAM.

Importare il modulo `pysam`

In [26]:
import pysam

## Come leggere gli allineamenti da un file `BAM`

`AlignmentFile` è la classe che rappresenta un set di allineamenti.

Un oggetto di tipo `AlignmentFile` può essere costruito a partire da un file SAM/BAM nel seguente modo: 

    samfile = pysam.AlignmentFile(sam_file_name, 'r')
    bamfile = pysam.AlignmentFile(bam_file_name, 'rb')

Importare la classe `AlignmentFile`.

### Costruire l'oggetto `AlignmentFile` a partire dal file `sample.bam`

Indicizzare il file `sample.bam` tramite la funzione `index()`.

Costruire l'oggetto `AlignmentFile`.

Il metodo `get_index_statistics()` fornisce qualche informazione generale.

In [31]:
bamfile.get_index_statistics()

[IndexStats(contig='X', mapped=25008, unmapped=0, total=25008)]

Il numero di reads mappati e non mappati si ottiene accedendo a `mapped` e `unmapped`:

Per ottenere le references coinvolte nel file BAM:

Per ottenere il numero di references coinvolte nel file:

Per ottenere le lunghezze delle references coinvolte nel file:

Per ottenere la lunghezza di una reference:

Per ottenere il numero di reads mappati a una determinata regione della reference:

## Come ottenere gli allineamenti dall'oggetto `AlignmentFile`

Le istruzioni: 

    bamfile.fetch()
    bamfile.fetch(ref_name)
    bamfile.fetch(ref_name, start_region, end_region)
    
restituiscono (rispettivamente) un iteratore contenente:
- tutti gli allineamenti del BAM file
- tutti gli allineamenti che si riferiscono alla reference `ref_name`
- tutti gli allineamenti che si riferiscono alla sottoregione tra le posizioni `start_region` e `end_region`della reference `ref_name`.

Un singolo allineamento tra query e reference è un oggetto di tipo `AlignedSegment`.

---
    
Si estraggano tutti gli allineamenti del BAM file.

Il numero di allineamenti estratti è:

Si estraggano ora tutti gli allineamenti relativa alla reference `X`.

Si estraggano infine tutti gli allineamenti relativa alla reference `X` che cadono nella regione tra le posizioni `280000` e `285000`.

L'istruzione:

    bamfile.head(n)

restituisce i primi `n` allineamenti del BAM file.

---

Si estraggano i primi 1000 allineamenti.

### Come manipolare gli allineamenti

Recuperare il primo allineamento della lista.

Gli oggetti `AlignedSegment` hanno una serie di attributi che descrivono l'allineamento.

Gli oggetti `AlignedSegment` hanno una serie di metodi tra i quali:

In [68]:
#first_alignment.get_reference_sequence()

## Qualche metodo di `AlignmentFile`

- `find_introns()` restituisce in un oggetto `Counter` gli introni e il loro supporto in termini di reads allineati

       bamfile.find_introns(alignment_iterator)

Per trovare tutti gli introni supportati dagli allineamenti del file in input basta scrivere:

- `pileup()`, restituisce un iteratore contenente un oggetto `PileupColumn` per ognuna delle basi coperte dagli allineamenti che cadono nella regione specificata come *argomento*.

       bamfile.pileup()
       bamfile.pileup(ref_name)
       bamfile.pileup(ref_name, start_region, end_region)

Ogni base della reference coperta da allineamenti viene chiamata *pileup column*.

---

Trovare tutte le basi della reference coperte dagli allineamenti presenti nel BAM file.

In [72]:
#help(pysam.libcalignedsegment.PileupColumn)

Gli allineamenti presenti nel BAM file coprono un totale di:

Accedere alla 1000-esima colonna di *pileup* (base della reference) e:

- settare la qualità minima a 0

- accedere alla posizione sulla reference

- recuperare il numero di allineamenti (segmenti) che coprono la base.

- ottenere gli identificatori dei reads (queries) che coprono la base.

- ottenere la lista degli allineamenti che coprono la base.

Per il primo oggetto `PileupRead`, accedere alla posizione sul read che corrisponde alla base di pileup sulla *reference*.

L'allineamento del primo read "impilato" è: